<a href="https://colab.research.google.com/github/kingketan9/BigDataLabs/blob/main/Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null ### Installing Java

In [2]:
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz #### Installing Spark and Hadoop on the worker node
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark #### Creating a Sparksession

1. Upload the dataset to Google Colab test.csv which describes the customer purchase behaviour

In [5]:
df = spark.read.csv('/content/cars (1).csv', header = True, sep = ";")

 2. Show Column details of the dataset

In [6]:
print("Column details:")
for col_name, col_type in df.dtypes:
    print(f"Column: {col_name}, Type: {col_type}")

Column details:
Column: Car, Type: string
Column: MPG, Type: string
Column: Cylinders, Type: string
Column: Displacement, Type: string
Column: Horsepower, Type: string
Column: Weight, Type: string
Column: Acceleration, Type: string
Column: Model, Type: string
Column: Origin, Type: string


3. Display the first 15 rows of the dataset

In [7]:
print("Top fifteen rows of the dataframe:")
df.show(15)

Top fifteen rows of the dataframe:
+--------------------+----+---------+------------+----------+------+------------+-----+------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|
+--------------------+----+---------+------------+----------+------+------------+-----+------+
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0| 3504.|        12.0|   70|    US|
|   Buick Skylark 320|15.0|        8|       350.0|     165.0| 3693.|        11.5|   70|    US|
|  Plymouth Satellite|18.0|        8|       318.0|     150.0| 3436.|        11.0|   70|    US|
|       AMC Rebel SST|16.0|        8|       304.0|     150.0| 3433.|        12.0|   70|    US|
|         Ford Torino|17.0|        8|       302.0|     140.0| 3449.|        10.5|   70|    US|
|    Ford Galaxie 500|15.0|        8|       429.0|     198.0| 4341.|        10.0|   70|    US|
|    Chevrolet Impala|14.0|        8|       454.0|     220.0| 4354.|         9.0|   70|    US|
|   Plymouth Fu

4. Print the total number of rows in the dataset

In [8]:
num_rows = df.count()
print("Total number of rows:", num_rows)

Total number of rows: 406


5. Determine the total Purchase per City Category

Note that you will have null values in your dataframe. Replace the null values with the mean of each column and find the above.

In [11]:
from pyspark.sql.functions import col, mean, when

In [ ]:
numeric_columns = [c for c, dtype in df.dtypes if dtype in ("int", "double", "float")]
column_means = df.select(*[mean(col(c)).alias(c) for c in numeric_columns]).first()

df_filled = df.na.fill(column_means.asDict())

purchase_per_category = df_filled.groupBy("City_Category").agg({"Purchase": "sum"})

purchase_per_category.show()

6. Remove rows which have missing values

In [9]:
df = df.dropna()

df.show()

+--------------------+----+---------+------------+----------+------+------------+-----+------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|
+--------------------+----+---------+------------+----------+------+------------+-----+------+
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0| 3504.|        12.0|   70|    US|
|   Buick Skylark 320|15.0|        8|       350.0|     165.0| 3693.|        11.5|   70|    US|
|  Plymouth Satellite|18.0|        8|       318.0|     150.0| 3436.|        11.0|   70|    US|
|       AMC Rebel SST|16.0|        8|       304.0|     150.0| 3433.|        12.0|   70|    US|
|         Ford Torino|17.0|        8|       302.0|     140.0| 3449.|        10.5|   70|    US|
|    Ford Galaxie 500|15.0|        8|       429.0|     198.0| 4341.|        10.0|   70|    US|
|    Chevrolet Impala|14.0|        8|       454.0|     220.0| 4354.|         9.0|   70|    US|
|   Plymouth Fury iii|14.0|        8|       440.0|